In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: QuikGraph, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-preview.17"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.17"

Installed Packages Newtonsoft.Json, 13.0.1 Plotly.NET, 2.0.0-preview.17 Plotly.NET.Interactive, 2.0.0-preview.17 QuikGraph, 2.3.0

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
#r "/Users/jnmatthews/MGGG/GerryChain.NET/GerryChain/bin/Release/net6.0/GerryChain.dll"

In [ ]:
using System;
using System.Linq;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using GerryChain;
using Plotly.NET;
using Plotly.NET.Interactive;
using Plotly.NET.LayoutObjects;

In [ ]:
var scores = Scores.MinShareOverThresholdPlusNextHighest("GinglesBVAP","BVAP", "VAP", 0.5);
var totSteps = 10000;
var scoreOpt = "GinglesBVAP";

var initPart = new Partition("../resources/al_vtds20_with_seeds.json", "CD_Seed", "TOTPOP", new string[] { "TOTPOP", "VAP", "BVAP" },
                             scores, regionAware: true, regionDivisionSpecs: new (string, double)[]{("COUNTY", 1)});

In [ ]:
var hotDur = 1000;
var coolDownDur = 0;
var coldDur = 4000;
var accept = AcceptanceFunctions.SimulatedAnnealingFactory(initPart, scoreOpt, hotDur, coolDownDur, coldDur, betaMagnitude: 500, minimize: false);
var chainAnneal = new Chain(initPart, totSteps, 0.01, batchSize:4, accept: accept);

In [ ]:
var scoresAnneal = new double[totSteps];
var maxesAnneal = new double[totSteps];
int i = 0;
double maxScore = 0;

foreach(Partition p in chainAnneal)
{
    double curScore = ((PlanWideScoreValue)p.Score(scoreOpt)).Value;
    if (curScore > maxScore) { maxScore = curScore; }
    scoresAnneal[i] = curScore;
    maxesAnneal[i] = maxScore;
    
    if(i % 1000 == 0) { Console.Write("*"); }
    i++;
}

**********

In [ ]:
var shortBursts = new ShortBurstOptimizer(initPart, 10, 1000, scoreOpt, 0.01, batchSize:4);

In [ ]:
var maxesBurst = new double[totSteps];
var scoresBurst = new double[totSteps];
int i = 0;

foreach(Partition p in shortBursts.Run())
{
    double curScore = ((PlanWideScoreValue)p.Score(scoreOpt)).Value;
    scoresBurst[i] = curScore;
    maxesBurst[i] = ((PlanWideScoreValue)shortBursts.BestScore).Value;
    
    if(i % 1000 == 0) { Console.Write("*"); }
    i++;
}

**********

In [ ]:
var yMin = scoresAnneal.Min()*0.95;
var yMax = scoresAnneal.Max()*1.05;
int cycleLength = hotDur + coolDownDur + coldDur;
int numCycles = totSteps / cycleLength;
var lines = new List<Shape>();

foreach(int i in Enumerable.Range(0, numCycles))
{
    int offset = cycleLength*i;
    lines.Add(Shape.init<int,int,double,double>(StyleParam.ShapeType.Line, hotDur+offset, hotDur+offset, yMin, yMax, Opacity: 0.3));
    lines.Add(Shape.init<int,int,double,double>(StyleParam.ShapeType.Line, hotDur+coolDownDur+offset, hotDur+coolDownDur+offset, yMin, yMax, Opacity: 0.3));
    lines.Add(Shape.init<int,int,double,double>(StyleParam.ShapeType.Line, cycleLength*(i+1), cycleLength*(i+1), yMin, yMax, Opacity: 0.3));
}

GenericChart.GenericChart chartScores = Chart2D.Chart.Line<int, double, int>(Enumerable.Range(0, totSteps), scoresAnneal, Name: "Raw Score")
    .WithXAxisStyle(title: Title.init("Step"), ShowGrid: false, ShowLine: true)
    .WithYAxisStyle(title: Title.init(scoreOpt), ShowGrid: false, ShowLine: true);
    

GenericChart.GenericChart chartMaxes = Chart2D.Chart.Line<int, double, int>(Enumerable.Range(0, totSteps), maxesAnneal, Name: "Max Score")
    .WithXAxisStyle(title: Title.init("Step"), ShowGrid: false, ShowLine: true)
    .WithYAxisStyle(title: Title.init(scoreOpt), ShowGrid: false, ShowLine: true);

Chart.Combine(new GenericChart.GenericChart[] {chartScores, chartMaxes})
    .WithShapes(lines)

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
var yMin = scoresBurst.Min()*0.95;
var yMax = maxesBurst.Max()*1.05;

GenericChart.GenericChart chartScores = Chart2D.Chart.Line<int, double, int>(Enumerable.Range(0, totSteps), scoresBurst, Name: "Raw Score")
    .WithXAxisStyle(title: Title.init("Step"), ShowGrid: false, ShowLine: true)
    .WithYAxisStyle(title: Title.init(scoreOpt), ShowGrid: false, ShowLine: true);

GenericChart.GenericChart chartMaxes = Chart2D.Chart.Line<int, double, int>(Enumerable.Range(0, totSteps), maxesBurst, Name: "Max Score")
    .WithXAxisStyle(title: Title.init("Step"), ShowGrid: false, ShowLine: true)
    .WithYAxisStyle(title: Title.init(scoreOpt), ShowGrid: false, ShowLine: true);

Chart.Combine(new GenericChart.GenericChart[] {chartScores, chartMaxes})

<!-- Plotly chart will be drawn inside this DIV -->